In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import open3d as o3d
from torch.utils.data import DataLoader
from learning3d.data_utils import ClassificationData, ModelNet40Data
from model_conv1d import Encoder, PPFFoldNet
import random
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ModelNet40 veri kümesini yükleme
train_dataset = ClassificationData(data_class=ModelNet40Data(train=True, num_points=2048, use_normals=False))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

test_dataset = ClassificationData(data_class=ModelNet40Data(train=False, num_points=2048, use_normals=False))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)


class PointCloudNormalizer(nn.Module):
    def __init__(self):
        super(PointCloudNormalizer, self).__init__()

    def forward(self, x):
        """
        Input: x -> (B, N, 6)  -> (x, y, z, nx, ny, nz)
        Output: out -> (B, N, 4) -> (x, y, z, norm_magnitude)
        """
        xyz = x[:, :, :3]  # (B, N, 3) - x, y, z koordinatları
        normals = x[:, :, 3:]  # (B, N, 3) - normal vektörleri
        
        # Normallerin büyüklüğünü hesapla (L2 normu)
        norm_magnitude = torch.norm(normals, dim=2, keepdim=True)  # (B, N, 1)

        # Yeni tensörü oluştur (x, y, z, norm büyüklüğü)
        out = torch.cat((xyz, norm_magnitude), dim=2)  # (B, N, 4)
        return out


class EncoderHead(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256, output_dim=128):
        super(EncoderHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.norm1 = nn.BatchNorm1d(hidden_dim)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.norm2 = nn.BatchNorm1d(output_dim)  # Yeni ekleme

    def forward(self, x):
        x = self.fc1(x)
        x = self.norm1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.norm2(x)  # Yeni ekleme
        return x



class PPFFoldNetEncoder(nn.Module):
    """Tüm modelleri birleştiren encoder"""
    def __init__(self, input_dim=6, hidden_dim=512, output_dim=128):
        super(PPFFoldNetEncoder, self).__init__()
        #self.normalizer = PointCloudNormalizer()
        self.base = Encoder(num_points_per_patch=2048)
        self.head = EncoderHead(512, 256, 128)

    def forward(self, x):
        #x = self.normalizer(x)
        x = self.base(x)
        x = torch.squeeze(x)
        x = self.head(x)
        return x.squeeze()

encoder_model = PPFFoldNetEncoder().to("cuda")


class PPFFoldNetClassifier(nn.Module):
    def __init__(self, num_classes=40, embedding_dim=128):
        super(PPFFoldNetClassifier, self).__init__()
        self.encoder = encoder_model
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )
    
    def forward(self, points):
        features = self.encoder(points)
        global_embedding = features.squeeze()
        logits = self.fc(global_embedding)
        return logits

model = PPFFoldNetEncoder().to(device)
model.load_state_dict(torch.load("ppffoldnet_encoder_epoch60.pth", map_location=device))
model.eval()

def random_rotation_single_axis(points, max_angle):
    """Belirli bir eksende rastgele dönüşüm uygular."""
    axis = random.choice([0, 1, 2])  # Rastgele bir eksen seç
    angle = random.uniform(-max_angle, max_angle)  # Rastgele dönüş açısı belirle
    
    rotation_matrix = torch.eye(3)  # 3x3 birim matris
    c, s = torch.cos(torch.tensor(angle)), torch.sin(torch.tensor(angle))
    
    if axis == 0:  # X ekseni dönüşü
        rotation_matrix[1, 1] = c
        rotation_matrix[1, 2] = -s
        rotation_matrix[2, 1] = s
        rotation_matrix[2, 2] = c
    elif axis == 1:  # Y ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 2] = s
        rotation_matrix[2, 0] = -s
        rotation_matrix[2, 2] = c
    else:  # Z ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 1] = -s
        rotation_matrix[1, 0] = s
        rotation_matrix[1, 1] = c

    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def random_rotation(points, max_angle):
    """Rastgele dönüşüm uygular."""
    angles = torch.empty(3).uniform_(-max_angle, max_angle)  # Her eksen için rastgele açı seç
    Rx = torch.tensor([
        [1, 0, 0],
        [0, torch.cos(angles[0]), -torch.sin(angles[0])],
        [0, torch.sin(angles[0]), torch.cos(angles[0])]
    ])
    Ry = torch.tensor([
        [torch.cos(angles[1]), 0, torch.sin(angles[1])],
        [0, 1, 0],
        [-torch.sin(angles[1]), 0, torch.cos(angles[1])]
    ])
    Rz = torch.tensor([
        [torch.cos(angles[2]), -torch.sin(angles[2]), 0],
        [torch.sin(angles[2]), torch.cos(angles[2]), 0],
        [0, 0, 1]
    ])
    rotation_matrix = torch.matmul(Rz, torch.matmul(Ry, Rx))  # Döndürme matrisini oluştur
    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def cosine_similarity(vec1, vec2):
    """Cosine similarity hesaplar."""
    return F.cosine_similarity(vec1, vec2, dim=-1).mean().item()

def euclidean_distance(vec1, vec2):
    """Euclidean distance hesaplar."""
    return torch.norm(vec1 - vec2, p=2, dim=-1).mean().item()

def evaluate_encoder_similarity():
    """Encoder çıktılarının benzerlik skorlarını hesaplar."""
    scores = {
        "Single Small": {"cosine": [], "euclidean": []},
        "All Small": {"cosine": [], "euclidean": []},
        "Single Large": {"cosine": [], "euclidean": []},
        "All Large": {"cosine": [], "euclidean": []},
    }

    with torch.no_grad():
        for points, _ in test_loader:
            points = points.to(device)

            # 4 Farklı Dönüşüm
            rotated_1 = random_rotation_single_axis(points, torch.pi / 18)  # Single Small (~10°)
            rotated_2 = random_rotation(points, torch.pi / 18)  # All Small (~10°)
            rotated_3 = random_rotation_single_axis(points, torch.pi / 3)   # Single Large (~60°)
            rotated_4 = random_rotation(points, torch.pi / 3)   # All Large (~60°)

            # **Encoder Çıktıları**
            original_encoding = model(points)
            original_encoding = torch.squeeze(original_encoding)
            enc_1 = model(rotated_1)
            enc_1 = torch.squeeze(enc_1)
            enc_2 = model(rotated_2)
            enc_2 = torch.squeeze(enc_2)
            enc_3 = model(rotated_3)
            enc_3 = torch.squeeze(enc_3)
            enc_4 = model(rotated_4)
            enc_4 = torch.squeeze(enc_4)

            scores["Single Small"]["cosine"].append(cosine_similarity(original_encoding, enc_1))
            scores["Single Small"]["euclidean"].append(euclidean_distance(original_encoding, enc_1))

            scores["All Small"]["cosine"].append(cosine_similarity(original_encoding, enc_2))
            scores["All Small"]["euclidean"].append(euclidean_distance(original_encoding, enc_2))

            scores["Single Large"]["cosine"].append(cosine_similarity(original_encoding, enc_3))
            scores["Single Large"]["euclidean"].append(euclidean_distance(original_encoding, enc_3))

            scores["All Large"]["cosine"].append(cosine_similarity(original_encoding, enc_4))
            scores["All Large"]["euclidean"].append(euclidean_distance(original_encoding, enc_4))

    # **Ortalama Skorları Hesapla ve Yazdır**
    print("\n==== Ortalama Benzerlik ve Mesafeler ====")
    for key, values in scores.items():
        avg_cosine = sum(values["cosine"]) / len(values["cosine"])
        avg_euclidean = sum(values["euclidean"]) / len(values["euclidean"])
        print(f"{key} - Cosine Similarity: {avg_cosine:.4f}, Euclidean Distance: {avg_euclidean:.4f}")


evaluate_encoder_similarity()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device: cuda

==== Ortalama Benzerlik ve Mesafeler ====
Single Small - Cosine Similarity: 0.9747, Euclidean Distance: 0.3726
All Small - Cosine Similarity: 0.9277, Euclidean Distance: 0.7032
Single Large - Cosine Similarity: 0.6129, Euclidean Distance: 1.5837
All Large - Cosine Similarity: 0.2339, Euclidean Distance: 2.5089
